# Notebook for Toronto Neighborhoods project

In [1]:
# imports
import pandas as pd
import lxml

### 1. Read table from wikipedia

In [36]:
# get table from wikipedia
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)
neigh = tables[0]
neigh.rename(columns = {'Postal Code': 'PostalCode'}, inplace=True)

In [37]:
# subset to Borough != 'Not assigned'
mask = neigh.Borough != 'Not assigned'
neigh = neigh[mask].reset_index(drop=True)
neigh.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [43]:
# check for Neighbourhood == 'Not assigned'
neigh[neigh.Neighbourhood == 'Not assigned'].shape
print('All Neighbourhoods assigned!')

All Neighbourhoods assigned


In [44]:
# check for duplicate PostalCodes
neigh['PostalCode'].value_counts().max()
print('No duplicate PostalCodes found!')

No duplicate PostalCodes found
